In [1]:
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import os
import numpy as np


print("Import cac thu vien")

Import cac thu vien


In [4]:

BASE_DIR = r"D:\BookInsight\data"
CLEAN_PARQUET_FILE = r"D:\BookInsight\data\demo_100k_samples\amazon_books_clean_100k_samples.parquet"


# SQL Path
SQL_DIR = os.path.join(BASE_DIR, "database")
SQL_DB_PATH = os.path.join(SQL_DIR, "books.db")
SCHEMA_PATH = os.path.join(BASE_DIR, "schema.sql")

# Vector Store Paths
VS_DIR = os.path.join(BASE_DIR, "vectorstores")
FAISS_INDEX_PATH = os.path.join(VS_DIR, "index.faiss")
EMBEDDINGS_PATH = os.path.join(VS_DIR, "embeddings.npy")
META_PATH = os.path.join(VS_DIR, "unique_ids.pkl")

os.makedirs(SQL_DIR, exist_ok=True)
os.makedirs(VS_DIR, exist_ok=True)

print("Cau hinh duong dan hoan tat.")
print(f"File sach dau vao:\n {CLEAN_PARQUET_FILE}")
print(f"File SQL dau ra:\n {SQL_DB_PATH}")
print(f"Folder VSS dau ra:\n {VS_DIR}")

Cau hinh duong dan hoan tat.
File sach dau vao:
 D:\BookInsight\data\demo_100k_samples\amazon_books_clean_100k_samples.parquet
File SQL dau ra:
 D:\BookInsight\data\database\books.db
Folder VSS dau ra:
 D:\BookInsight\data\vectorstores


In [5]:
print(f"Đang đọc file: {CLEAN_PARQUET_FILE}...")

try:
    df_final = pd.read_parquet(CLEAN_PARQUET_FILE)
    print(f"Đọc xong {len(df_final)} dòng.")
    df_final.info()
except Exception as e:
    print(f"Lỗi khi đọc file Parquet: {e}")
    print("Vui lòng kiểm tra lại đường dẫn file CLEAN_PARQUET_FILE ở Cell 2.")

Đang đọc file: D:\BookInsight\data\demo_100k_samples\amazon_books_clean_100k_samples.parquet...
Đọc xong 100000 dòng.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   unique_id         100000 non-null  object 
 1   title             100000 non-null  object 
 2   description       100000 non-null  object 
 3   features          100000 non-null  object 
 4   author_name       100000 non-null  object 
 5   author_about      100000 non-null  object 
 6   author_avatar     71416 non-null   object 
 7   publisher         100000 non-null  object 
 8   publication_year  100000 non-null  int64  
 9   main_category     100000 non-null  object 
 10  categories        100000 non-null  object 
 11  book_format       100000 non-null  object 
 12  language          100000 non-null  object 
 13  page_count        100000 non-null  int64  
 14 

In [6]:
# --- 2. Xây dựng SQL Database (Cho SQLTool) ---
print(f"Bắt đầu xây dựng SQL Database tại: {SQL_DB_PATH}...")

sql_columns = [
    'unique_id', 'title', 'author_name', 'publisher', 'publication_year',
    'main_category', 'categories', 'book_format', 'language', 'page_count',
    'isbn_13', 'price', 'average_rating', 'rating_number',
    'main_images', 'author_avatar'
]
df_sql = df_final[sql_columns].copy()

try:
    with open(SCHEMA_PATH, 'r', encoding='utf-8') as f:
        schema_script = f.read()

    conn = sqlite3.connect(SQL_DB_PATH)
    cursor = conn.cursor()
    print(f"Đang thực thi schema.sql (tạo bảng và indexes) tại {SQL_DB_PATH}...")
    cursor.executescript(schema_script)

    # Nạp dữ liệu vào bảng 'books'
    print(f"Đang nạp {len(df_sql)} dòng vào bảng 'books'...")
    # Dùng 'append' vì bảng đã được tạo (và trống) bởi schema.sql
    df_sql.to_sql('books', conn, if_exists='append', index=False)

    conn.commit()
    conn.close()

    print("Xây dựng SQL Database và Indexes hoàn tất.")
except Exception as e:
    print(f"Lỗi xây dựng SQL: {e}")

Bắt đầu xây dựng SQL Database tại: D:\BookInsight\data\database\books.db...
Đang thực thi schema.sql (tạo bảng và indexes) tại D:\BookInsight\data\database\books.db...
Đang nạp 100000 dòng vào bảng 'books'...
Xây dựng SQL Database và Indexes hoàn tất.


In [9]:
# Chuẩn bị Dữ liệu Vector (Cho RAGTool) ---
print("Bắt đầu chuẩn bị dữ liệu cho Vector Store...")
vector_columns = ['unique_id', 'title', 'description', 'features', 'author_about']
df_vector = df_final[vector_columns].copy()

def create_rag_document(row):
    title = str(row['title'] or '')
    author_bio = str(row['author_about'] or '')
    description = str(row['description'] or '')
    features = str(row['features'] or '')

    return f"Title: {title}\nAuthor Bio: {author_bio}\nDescription: {description}\nFeatures: {features}"

df_vector['rag_document_text'] = df_vector.apply(create_rag_document, axis=1)

print("Chuẩn bị dữ liệu Vector hoàn tất.")
print("--- 1 document mẫu ---")
print(df_vector['rag_document_text'].iloc[1])

Bắt đầu chuẩn bị dữ liệu cho Vector Store...
Chuẩn bị dữ liệu Vector hoàn tất.
--- 1 document mẫu ---
Title: Notes from a Kidwatcher
Author Bio: Discover more of the author’s books, see similar authors, read author blogs and more
Description: About the Author. SANDRA WILDE, Ph.D., is widely recognized for her expertise in developmental spelling and her advocacy of holistic approaches to spelling and phonics. She is Professor of Curriculum and Instruction at Portland State University in Oregon. She is best known for her work in invented spelling, phonics and miscue analysis. She specializes in showing teachers how kids' invented spellings and miscues can help us work with them in more sophisticated and learner-centered ways. Looking at what kids do as they read and write is at the heart of Sandra's presentations and workshops. She can do lively keynote presentations that highlight the interesting things that we can learn by paying close attention to students' invented spellings and misc

In [10]:
try:
    model_name = "BAAI/bge-small-en-v1.5"
    print(f"Đang tải Embedding Model ({model_name})...")

    model = SentenceTransformer(model_name)
    print("Tải Model hoàn tất.")

except Exception as e:
    print(f" Lỗi tải model: {e}")

Đang tải Embedding Model (BAAI/bge-small-en-v1.5)...
Tải Model hoàn tất.


In [11]:
print("Bắt đầu xây dựng Vector Store (FAISS)...")
try:
    print(f"Bắt đầu embedding {len(df_vector)} documents với model BGE...")
    documents_to_embed = df_vector['rag_document_text'].tolist()


    embeddings = model.encode(documents_to_embed, show_progress_bar=True)
    embeddings = np.array(embeddings).astype('float32')
    print("Embedding hoàn tất.")


    d = embeddings.shape[1] # Dimension (sẽ tự động là 384 cho BGE-small)
    index = faiss.IndexFlatL2(d)
    print(f"Đang add {len(embeddings)} vectors (kích thước {d}) vào FAISS index...")
    index.add(embeddings)

    # Lưu 3 file ra đĩa (FAISS_INDEX_PATH, EMBEDDINGS_PATH, META_PATH)
    print(f"Đang lưu FAISS index ra đĩa tại: {FAISS_INDEX_PATH}...")
    faiss.write_index(index, FAISS_INDEX_PATH)

    print(f"Đang lưu embeddings.npy tại: {EMBEDDINGS_PATH}")
    np.save(EMBEDDINGS_PATH, embeddings)

    print(f"Đang lưu unique_ids.pkl tại: {META_PATH}")
    with open(META_PATH, 'wb') as f:
        pickle.dump(df_vector['unique_id'].tolist(), f)

    print("\n" + "="*30)
    print(f"GIAI ĐOẠN 1 HOÀN TẤT")
    print(f"Đã lưu thành công SQL DB và Vector Store tại: {BASE_DIR}")
    print("="*30)

except Exception as e:
    print(f"Lỗi xây dựng Vector Store: {e}")

Bắt đầu xây dựng Vector Store (FAISS)...
Bắt đầu embedding 100000 documents với model BGE...


Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

Embedding hoàn tất.
Đang add 100000 vectors (kích thước 384) vào FAISS index...
Đang lưu FAISS index ra đĩa tại: D:\BookInsight\data\vectorstores\index.faiss...
Đang lưu embeddings.npy tại: D:\BookInsight\data\vectorstores\embeddings.npy
Đang lưu unique_ids.pkl tại: D:\BookInsight\data\vectorstores\unique_ids.pkl

GIAI ĐOẠN 1 HOÀN TẤT
Đã lưu thành công SQL DB và Vector Store tại: D:\BookInsight\data
